In [2]:
# !pip install langgraph-supervisor -q
# install package
# !pip install -U langchain-ollama -q
# !pip install langchain -q
# !pip install -U langchain-community -q
# !pip install numexpr -q
# !pip install -U duckduckgo-search -q
# !pip install google-search-results>=2.4.2 -q
# !pip install -qU langchain-tavily -q
# !pip install streamlit -q
# !pip install pandas matplotlib seaborn -q
# !pip install networkx pyvis streamlit -q
# !pip install crewai-tools -q

In [16]:
import os
from dotenv import load_dotenv
load_dotenv() 

True

In [17]:

os.environ["TAVILY_API_KEY"] = os.getenv("API_KEY")

In [19]:
from langchain_ollama import ChatOllama
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# Import actual tool integrations
from langchain_tavily import TavilySearch
from langchain_community.tools.google_trends import GoogleTrendsQueryRun
from langchain_community.utilities.google_trends import GoogleTrendsAPIWrapper



In [25]:
# Set up the local Ollama model (qwen2.5 - 14b)
local_llm = "qwen2.5:14b"
model = ChatOllama(model=local_llm, temperature=0.0)

In [26]:
model.invoke('whos the president of the united states?')

AIMessage(content='As of my last update in September 2023, the President of the United States is Joe Biden, who took office on January 20, 2021. However, to get the most current information, you might want to check a reliable news source or official government websites.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:14b', 'created_at': '2025-03-23T17:15:34.8951587Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21646490600, 'load_duration': 20856829500, 'prompt_eval_count': 38, 'prompt_eval_duration': 92000000, 'eval_count': 62, 'eval_duration': 546000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a851366e-a611-49f2-9041-423358dd9cee-0', usage_metadata={'input_tokens': 38, 'output_tokens': 62, 'total_tokens': 100})

In [22]:
tavily_search_tool = TavilySearch()     # Performs real DuckDuckGo searches
google_trends_tool = GoogleTrendsQueryRun(api_wrapper=GoogleTrendsAPIWrapper())        # Analyzes trends for given topics



In [23]:
tavily_search_tool.invoke("whos the president of the united states?")

{'query': 'whos the president of the united states?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'List of presidents of the United States - Wikipedia',
   'url': 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States',
   'content': 'The president of the United States is the head of state and head of government of the United States,[1] indirectly elected to a four-year term via the Electoral College.[2] Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.[3] The first president, George Washington, won a unanimous vote of the Electoral College.[4] The incumbent president is Donald Trump, who assumed office on January 20, 2025.[5][6] Since the office was established in 1789, 45 men have served in 47 presidencies; the discrepancy arises from two individuals elected to non-consecutive terms: Grover Cleveland is counted as t

In [7]:
research_agent = create_react_agent(
    model=model,
    # tools=[ddg_search_tool, google_trends_tool],
    tools=[],
    name="research_expert",
    prompt=(
        "You are a research expert. Use DuckDuckGo and Google Trends to collect accurate, up-to-date information. "
        "Do not write summaries or outlines."
    )
)

In [8]:
outline_agent = create_react_agent(
    model=model,
    tools=[],
    name="outline_expert",
    prompt=(
        "You are an expert at creating structured outlines. Based on the research, create a well-organized outline "
        "for an article with appropriate headings and bullet points."
    )
)

In [9]:
#  Drafting agent using LLM only
drafting_agent = create_react_agent(
    model=model,
    tools=[],
    name="drafting_expert",
    prompt=(
        "You are a skilled content writer. Expand the outline into a full article. Ensure clear structure and flow."
    )
)

In [10]:
# Editing agent using LLM only
editing_agent = create_react_agent(
    model=model,
    tools=[],
    name="editing_expert",
    prompt=(
        "You are a professional editor. Improve grammar, flow, and clarity of the article while maintaining the message."
    )
)

In [11]:
#  SEO agent using LLM only
seo_agent = create_react_agent(
    model=model,
    tools=[],
    name="seo_expert",
    prompt=(
        "You are an SEO specialist. Suggest keyword improvements, optimize headings, and write a strong meta description."
    )
)

In [12]:
# Create supervisor workflow
workflow = create_supervisor(
    agents=[research_agent, outline_agent, drafting_agent, editing_agent, seo_agent],
    model=model,
    prompt=(
        "You are the supervisor of a content generation team. Complete the following:\n"
        "1. Research: Use research_expert to collect insights.\n"
        "2. Outline: Use outline_expert to turn the research into a structured plan.\n"
        "3. Drafting: Use drafting_expert to write the article.\n"
        "4. Editing: Use editing_expert to polish the writing.\n"
        "5. SEO: Use seo_expert to enhance it for search visibility.\n\n"
        "Return the final optimized article."
    ),
    output_mode="full_history"
)

In [13]:
# # Compile and run
# checkpointer = InMemorySaver()
# store = InMemoryStore()
# app = workflow.compile(checkpointer=checkpointer, store=store)


In [15]:

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Produce an article on the impact of AI on content creation."
        }
    ]
})

ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'transfer_to_outline_expert', 'args': {}, 'id': 'd11225a4-19f2-4fce-8701-593bd8b383c4', 'type': 'tool_call'}, {'name': 'transfer_to_drafting_expert', 'args': {}, 'id': '77c979bc-1ec9-48d7-a4b0-d57621e42a51', 'type': 'tool_call'}, {'name': 'transfer_to_editing_expert', 'args': {}, 'id': '7b409cb2-8816-47eb-83bd-884e939aa5a4', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY

In [ ]:
# from uuid import uuid4

# thread_id = str(uuid4())

# inputs = {
#     "messages": [
#         {
#             "role": "user",
#             "content": "Produce an article on the impact of AI on content creation."
#         }
#     ]
# }

# # ✅ Thread-level persistence config
# config = {
#     "configurable": {
#         "thread_id": thread_id  # Required for checkpointer
#     }
# }

# # ✅ Final invocation (two arguments)
# result = app.invoke(inputs, config=config)


ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'transfer_to_outline_expert', 'args': {}, 'id': '7d600e72-7625-4136-bf2c-0e62c47137fc', 'type': 'tool_call'}, {'name': 'transfer_to_drafting_expert', 'args': {}, 'id': 'e05b1dae-545f-4e76-818e-024906bdebb9', 'type': 'tool_call'}, {'name': 'transfer_to_editing_expert', 'args': {}, 'id': 'b3c4be6f-b758-4596-ad5e-335b50442a11', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY

In [31]:

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# def web_search(query: str) -> str:
#     """Search the web for information."""
#     return (
#         "Here are the headcounts for each of the FAANG companies in 2024:\n"
#         "1. **Facebook (Meta)**: 67,317 employees.\n"
#         "2. **Apple**: 164,000 employees.\n"
#         "3. **Amazon**: 1,551,000 employees.\n"
#         "4. **Netflix**: 14,000 employees.\n"
#         "5. **Google (Alphabet)**: 181,269 employees."
#     )

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For general questions, use research_agent. "
        "For math problems, use math_agent."
    )
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "whos the president of america?"
        }
    ]
})

In [32]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

whos the president of america?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (3eaa4b4d-34f2-49ab-8885-7b84d84e9810)
 Call ID: 3eaa4b4d-34f2-49ab-8885-7b84d84e9810
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

The current President of the United States is Joe Biden, who took office on January 20, 2021.
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (18c7ef7d-1710-40d3-bde4-1090cd164791)
 Call ID: 18c7ef7d-1710-40d3-bde4-1090cd164791
  Args:
=======================